In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path = r'd:\Documentos\Projects\Python\EMDR-Project\src\sensor\data\sensor_data_20250611_195404.csv'

In [5]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(path, 
                 encoding='utf-8',  # Si hay caracteres especiales
                 index_col=0)  # Si la primera columna es el índice

# Mostrar información básica del DataFrame
print("Información del DataFrame:")
print(df.info())
print("\nPrimeras 5 filas:")
print(df.head())
print("\nÚltimas 5 filas:")
print(df.tail())
print(f"\nForma del DataFrame: {df.shape}")
print(f"\nTiempo total:")
# Convertir milisegundos a segundos y minutos
milisegundos = 1924136

# Convertir a segundos
segundos_total = milisegundos / 1000
print(f"Total en segundos: {segundos_total}")

# Convertir a minutos y segundos
minutos = int(segundos_total // 60)
segundos = segundos_total % 60

print(f"Tiempo: {minutos} minutos y {segundos:.3f} segundos")

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 240518 entries, 0 to 240517
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Timestamp_ms  240518 non-null  int64  
 1   EOG_raw       240518 non-null  int64  
 2   EOG_filtrado  240518 non-null  float64
 3   PPG_raw       240518 non-null  int64  
 4   PPG_filtrado  240518 non-null  float64
 5   BPM           240518 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 12.8 MB
None

Primeras 5 filas:
    Timestamp_ms  EOG_raw  EOG_filtrado  PPG_raw  PPG_filtrado  BPM
ID                                                                 
0              0      611      2.119196   -12662    -10.210935  0.0
1              8    -2097      9.909152   -15130    -83.490696  0.0
2             16    -3140     -6.843396   -16798   -337.215300  0.0
3             24    -1581   -173.013615   -17906   -917.829791  0.0
4             32    -5720   -665.

In [9]:
def plot_eog_section(inicio_segundos=0):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Filtrar datos para ventana de 30 segundos
    fin_segundos = inicio_segundos + 30
    mask = (df['segundos'] >= inicio_segundos) & (df['segundos'] <= fin_segundos)
    datos_ventana = df[mask]
    
    ax.plot(datos_ventana['segundos'], datos_ventana['PPG_raw'], 'b-', linewidth=0.8)
    ax.set_xlabel('Tiempo (segundos)')
    ax.set_ylabel('PPG Raw')
    ax.set_title(f'PPG Raw - Segundos {inicio_segundos} a {fin_segundos}')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(inicio_segundos, fin_segundos)
    
    plt.tight_layout()
    plt.show()

 Bases de datos

In [1]:
import sys
import os

# Agregar el directorio raíz del proyecto al path
project_root = r'd:\Documentos\Projects\Python\EMDR-Project'
if project_root not in sys.path:
    sys.path.append(project_root)

# Importar el DatabaseManager
from src.database.database_manager import DatabaseManager
import pandas as pd

In [2]:
# Verificar conexión y mostrar estructura de las tablas
print("=== ESTRUCTURA DE LA BASE DE DATOS ===\n")

# Obtener todos los pacientes
pacientes = DatabaseManager.get_all_patients()
print(f"📋 PACIENTES ({len(pacientes)} registros):")
print("-" * 80)

if pacientes:
    df_pacientes = pd.DataFrame(pacientes)
    print(df_pacientes.to_string(index=False))
    print("\nColumnas disponibles:", list(df_pacientes.columns))
else:
    print("No hay pacientes registrados")

print("\n" + "="*80 + "\n")

=== ESTRUCTURA DE LA BASE DE DATOS ===

📋 PACIENTES (3 registros):
--------------------------------------------------------------------------------
 id apellido_paterno apellido_materno nombre fecha_nacimiento  celular      fecha_registro      comentarios
  1          Serrudo          Careaga   Eloy       2002-01-01 78337255 2025-06-13 06:15:01           Prueba
  2                x                x Carlos       2002-01-01 12345678 2025-06-13 06:15:01 Segundo paciente
  3                x                x    seb       2002-01-01 12345678 2025-06-13 06:15:01                x

Columnas disponibles: ['id', 'apellido_paterno', 'apellido_materno', 'nombre', 'fecha_nacimiento', 'celular', 'fecha_registro', 'comentarios']




In [ ]:
# Obtener todas las sesiones (necesitamos iterar por cada paciente)
print(f"🔬 SESIONES:")
print("-" * 80)

all_sessions = []

if pacientes:
    for paciente in pacientes:
        patient_id = paciente['id']
        patient_name = f"{paciente['apellido_paterno']} {paciente['apellido_materno']} {paciente['nombre']}"
        
        # Obtener sesiones del paciente
        sesiones = DatabaseManager.get_sessions_for_patient(patient_id)
        
        # Agregar información del paciente a cada sesión
        for sesion in sesiones:
            sesion['paciente_id'] = patient_id
            sesion['paciente_nombre'] = patient_name
            all_sessions.append(sesion)

if all_sessions:
    df_sesiones = pd.DataFrame(all_sessions)
    print(f"Total de sesiones: {len(all_sessions)}")
    print(df_sesiones.to_string(index=False))
    print("\nColumnas disponibles:", list(df_sesiones.columns))
else:
    print("No hay sesiones registradas")

print("\n" + "="*80 + "\n")

🔬 SESIONES:
--------------------------------------------------------------------------------
Total de sesiones: 3
 id               fecha                                                                   comentarios  paciente_id      paciente_nombre
  1 2025-05-26 16:41:51  Sesión actualizada el 2025-05-26 16:43:23. Datos almacenados: 5545 muestras.            1 Serrudo Careaga Eloy
  2 2025-05-28 19:16:09 Sesión actualizada el 2025-05-28 19:16:13. Datos almacenados: 18895 muestras.            2           x x Carlos
  3 2025-05-29 10:53:39 Sesión actualizada el 2025-05-29 10:53:43. Datos almacenados: 15646 muestras.            3              x x seb

Columnas disponibles: ['id', 'fecha', 'comentarios', 'paciente_id', 'paciente_nombre']




In [6]:
import numpy as np
import pickle
import zlib

In [7]:
def generar_timestamps_ms(num_muestras, frecuencia_hz=125):
    """
    Genera timestamps en milisegundos comenzando desde 0
    
    Args:
        num_muestras: Número total de muestras
        frecuencia_hz: Frecuencia de muestreo (125 Hz por defecto)
    
    Returns:
        numpy array con timestamps en milisegundos
    """
    # Calcular intervalo entre muestras en milisegundos
    intervalo_ms = 1000 / frecuencia_hz  # 1000ms / 125Hz = 8ms entre muestras
    
    # Generar array de timestamps comenzando en 0
    timestamps = np.arange(num_muestras) * intervalo_ms
    
    return timestamps.astype(np.int64)

def comprimir_datos(datos):
    """Comprime datos usando pickle + zlib"""
    pickled = pickle.dumps(datos)
    compressed = zlib.compress(pickled)
    return compressed

In [8]:
# Datos de las sesiones existentes
sesiones_info = [
    {"id": 1, "muestras": 5545},
    {"id": 2, "muestras": 18895}, 
    {"id": 3, "muestras": 15646}
]

In [10]:
print("🔄 Generando timestamps para las sesiones (comenzando en 0)...")
print("-" * 60)

for sesion in sesiones_info:
    session_id = sesion["id"]
    num_muestras = sesion["muestras"]
    
    # Generar el arreglo de timestamps comenzando en 0
    timestamps_ms = generar_timestamps_ms(num_muestras, 125)
    
    # Comprimir el arreglo
    datos_comprimidos = comprimir_datos(timestamps_ms)
    
    # Actualizar solo la columna datos_ms
    success = DatabaseManager.update_session(
        session_id=session_id,
        datos_ms=datos_comprimidos
    )
    
    if success:
        duracion_ms = timestamps_ms[-1]  # Último timestamp = duración total
        duracion_segundos = duracion_ms / 1000
        duracion_minutos = duracion_segundos / 60
        
        print(f"✅ Sesión {session_id}:")
        print(f"   - Muestras: {num_muestras:,}")
        print(f"   - Duración: {duracion_minutos:.2f} minutos ({duracion_segundos:.1f} segundos)")
        print(f"   - Primer timestamp: {timestamps_ms[0]} ms")
        print(f"   - Último timestamp: {timestamps_ms[-1]} ms")
        print(f"   - Intervalo: 8.0 ms entre muestras")
    else:
        print(f"❌ Error actualizando sesión {session_id}")

🔄 Generando timestamps para las sesiones (comenzando en 0)...
------------------------------------------------------------
✅ Sesión 1:
   - Muestras: 5,545
   - Duración: 0.74 minutos (44.4 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 44352 ms
   - Intervalo: 8.0 ms entre muestras
✅ Sesión 2:
   - Muestras: 18,895
   - Duración: 2.52 minutos (151.2 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 151152 ms
   - Intervalo: 8.0 ms entre muestras
✅ Sesión 3:
   - Muestras: 15,646
   - Duración: 2.09 minutos (125.2 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 125160 ms
   - Intervalo: 8.0 ms entre muestras


In [ ]:
print("🔍 Verificando datos guardados...")
print("-" * 60)

# Verificar que los datos se guardaron correctamente
for sesion in sesiones_info:
    session_id = sesion["id"]
    
    # Recuperar datos de la sesión
    session_data = DatabaseManager.get_session(session_id=session_id, signal_data=True)
    
    if session_data and session_data.get("datos_ms") is not None:
        ms_data = session_data["datos_ms"]
        print(f"✅ Sesión {session_id} - Datos recuperados correctamente:")
        print(f"   - Número de timestamps: {len(ms_data):,}")
        print(f"   - Primer timestamp: {ms_data[0]} ms")
        print(f"   - Último timestamp: {ms_data[-1]} ms")
        print(f"   - Duración total: {ms_data[-1]/1000:.1f} segundos")
        print(f"   - Diferencia promedio: {np.mean(np.diff(ms_data)):.1f} ms")
    else:
        print(f"❌ Sesión {session_id} - Error recuperando datos")
    
    print()

print("📊 Resumen:")
print("-" * 60)
total_muestras = sum(s["muestras"] for s in sesiones_info)
print(f"Total de muestras procesadas: {total_muestras:,}")
print(f"Frecuencia de muestreo: 125 Hz")
print(f"Intervalo entre muestras: 8 ms")
print("Timestamps generados comenzando en 0 ms para cada sesión")

🔍 Verificando datos guardados...
------------------------------------------------------------
✅ Sesión 1 - Datos recuperados correctamente:
   - Número de timestamps: 5,545
   - Primer timestamp: 0 ms
   - Último timestamp: 44352 ms
   - Duración total: 44.4 segundos
   - Diferencia promedio: 8.0 ms

✅ Sesión 2 - Datos recuperados correctamente:
   - Número de timestamps: 18,895
   - Primer timestamp: 0 ms
   - Último timestamp: 151152 ms
   - Duración total: 151.2 segundos
   - Diferencia promedio: 8.0 ms

✅ Sesión 3 - Datos recuperados correctamente:
   - Número de timestamps: 15,646
   - Primer timestamp: 0 ms
   - Último timestamp: 125160 ms
   - Duración total: 125.2 segundos
   - Diferencia promedio: 8.0 ms

📊 Resumen:
------------------------------------------------------------
Total de muestras procesadas: 40,086
Frecuencia de muestreo: 125 Hz
Intervalo entre muestras: 8 ms
Timestamps generados comenzando en 0 ms para cada sesión


HEART RATE

In [22]:
# Leer el CSV con el problema de desplazamiento de columnas
path = r'C:\Users\Eloy\Documents\PythonProjects\EMDR-Project\src\com.samsung.shealth.exercise.recovery_heart_rate.20250630153438.csv'

# Primero, leer solo las columnas (línea 2)
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    header_line = lines[1].strip()  # Línea 2 (índice 1) tiene los nombres de columnas
    data_lines = lines[2:]  # Línea 3 en adelante son los datos

# Dividir los nombres de columnas
column_names = header_line.split(',')
# column_names.pop(0)  # Eliminar la primera columna vacía
print("Nombres de columnas encontrados:")
for i, col in enumerate(column_names):
    print(f"{i}: '{col}'")

Nombres de columnas encontrados:
0: 'create_sh_ver'
1: 'start_time'
2: 'modify_sh_ver'
3: 'update_time'
4: 'create_time'
5: 'time_offset'
6: 'deviceuuid'
7: 'exercise_id'
8: 'pkg_name'
9: 'end_time'
10: 'datauuid'
11: 'heart_rate'


In [23]:
# Opción 2: Usar usecols para especificar qué columnas leer (excluyendo la primera)
heart_rate_df = pd.read_csv(
    path,
    skiprows=2,  # Saltar las primeras 2 líneas
    header=None,  # No usar header automático
    usecols=range(1, len(column_names))  # Leer desde la columna 1 en adelante
)

# Los nombres de columnas también deben ajustarse
column_names_clean = column_names[1:]  # Quitar el primer nombre 'source'
heart_rate_df.columns = column_names_clean

print("\nDataFrame sin la columna 'source':")
heart_rate_df.tail()


DataFrame sin la columna 'source':


,start_time,modify_sh_ver,update_time,create_time,time_offset,deviceuuid,exercise_id,pkg_name,end_time,datauuid,heart_rate
0,2025-06-30 19:31:42.054,62980011,2025-06-30 19:33:43.111,2025-06-30 19:33:43.111,UTC-0400,a0CzBME90p,2f52f76d-ec1b-483e-8e72-c34a9212cac1,com.sec.android.app.shealth,2025-06-30 19:33:42.054,1c0e4448-f8fb-45f4-8b50-a949b7981fe8,1c0e4448-f8fb-45f4-8b50-a949b7981fe8.heart_rat...


In [19]:
# Convertir tipos de datos después de cargar
# Convertir a float las columnas numéricas
numeric_columns = [
    'com.samsung.health.heart_rate.heart_rate',
    'com.samsung.health.heart_rate.max', 
    'com.samsung.health.heart_rate.min',
    'com.samsung.health.heart_rate.heart_beat_count'
]

for col in numeric_columns:
    if col in heart_rate_df.columns:
        heart_rate_df[col] = pd.to_numeric(heart_rate_df[col], errors='coerce')

# Convertir a datetime las columnas de fecha
date_columns = [
    'com.samsung.health.heart_rate.start_time',
    'com.samsung.health.heart_rate.update_time', 
    'com.samsung.health.heart_rate.create_time',
    'com.samsung.health.heart_rate.end_time'
]

for col in date_columns:
    if col in heart_rate_df.columns:
        heart_rate_df[col] = pd.to_datetime(heart_rate_df[col], errors='coerce')

print("✅ Conversión de tipos completada")
print("\nInformación del DataFrame:")
print(heart_rate_df.info())

print("\nVerificar conversiones:")
print(f"Tipo de heart_rate: {heart_rate_df['com.samsung.health.heart_rate.heart_rate'].dtype}")
print(f"Tipo de start_time: {heart_rate_df['com.samsung.health.heart_rate.start_time'].dtype}")

print("\nPrimeras filas con tipos convertidos:")
heart_rate_df.head()

✅ Conversión de tipos completada

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 20 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   tag_id                                          1133 non-null   int64         
 1   com.samsung.health.heart_rate.create_sh_ver     1133 non-null   int64         
 2   com.samsung.health.heart_rate.heart_beat_count  1133 non-null   int64         
 3   com.samsung.health.heart_rate.start_time        1133 non-null   datetime64[ns]
 4   com.samsung.health.heart_rate.custom            0 non-null      float64       
 5   com.samsung.health.heart_rate.binning_data      27 non-null     object        
 6   com.samsung.health.heart_rate.modify_sh_ver     1133 non-null   int64         
 7   com.samsung.health.heart_rate.update_time       1133 non-null   datetim

,tag_id,com.samsung.health.heart_rate.create_sh_ver,com.samsung.health.heart_rate.heart_beat_count,com.samsung.health.heart_rate.start_time,com.samsung.health.heart_rate.custom,com.samsung.health.heart_rate.binning_data,com.samsung.health.heart_rate.modify_sh_ver,com.samsung.health.heart_rate.update_time,com.samsung.health.heart_rate.create_time,com.samsung.health.heart_rate.client_data_id,com.samsung.health.heart_rate.max,com.samsung.health.heart_rate.min,com.samsung.health.heart_rate.client_data_ver,com.samsung.health.heart_rate.time_offset,com.samsung.health.heart_rate.deviceuuid,com.samsung.health.heart_rate.comment,com.samsung.health.heart_rate.pkg_name,com.samsung.health.heart_rate.end_time,com.samsung.health.heart_rate.datauuid,com.samsung.health.heart_rate.heart_rate
0,21312,62980011,1,2025-05-30 12:50:48.616,NaN,NaN,62980011,2025-05-30 12:52:37.313,2025-05-30 12:52:37.313,NaN,NaN,NaN,NaN,UTC-0400,a0CzBME90p,NaN,com.sec.android.app.shealth,2025-05-30 12:50:48.616,54fc84af-9d5c-4e0c-be00-9136f930aeeb,85.0
1,21312,62980011,1,2025-05-30 13:00:29.290,NaN,NaN,62980011,2025-05-30 13:06:34.155,2025-05-30 13:06:34.155,NaN,NaN,NaN,NaN,UTC-0400,a0CzBME90p,NaN,com.sec.android.app.shealth,2025-05-30 13:00:29.290,b7eb787a-01bb-4734-ae59-20d68e257f47,82.0
2,21312,62980011,1,2025-05-30 13:11:00.876,NaN,NaN,62980011,2025-05-30 13:11:58.142,2025-05-30 13:11:58.142,NaN,NaN,NaN,NaN,UTC-0400,a0CzBME90p,NaN,com.sec.android.app.shealth,2025-05-30 13:11:00.876,973f9272-6096-4a9e-9d6b-8a5ba04b95aa,98.0
3,21312,62980011,1,2025-05-30 13:30:29.280,NaN,NaN,62980011,2025-05-30 13:34:59.153,2025-05-30 13:34:59.153,NaN,NaN,NaN,NaN,UTC-0400,a0CzBME90p,NaN,com.sec.android.app.shealth,2025-05-30 13:30:29.280,eefb229b-6704-4a9b-a482-67cdb84ba8d6,82.0
4,21312,62980011,1,2025-05-30 13:40:29.267,NaN,NaN,62980011,2025-05-30 13:49:45.763,2025-05-30 13:49:45.763,NaN,NaN,NaN,NaN,UTC-0400,a0CzBME90p,NaN,com.sec.android.app.shealth,2025-05-30 13:40:29.267,e734ffa5-75d5-4ab6-a6e7-9debd6bdef59,78.0


In [20]:
# Seleccionar solo las columnas que te interesan para la contrastación
# Renombrar para mayor comodidad (opcional, pero recomendado)
df_pulso = heart_rate_df[[
    'com.samsung.health.heart_rate.start_time',
    'com.samsung.health.heart_rate.end_time',
    'com.samsung.health.heart_rate.heart_rate',
    'com.samsung.health.heart_rate.min',
    'com.samsung.health.heart_rate.max',
    'com.samsung.health.heart_rate.time_offset'
]].copy() # Usar .copy() para evitar SettingWithCopyWarning

df_pulso = df_pulso.rename(columns={
    'com.samsung.health.heart_rate.start_time': 'tiempo_inicio',
    'com.samsung.health.heart_rate.end_time': 'tiempo_fin',
    'com.samsung.health.heart_rate.heart_rate': 'pulso_bpm',
    'com.samsung.health.heart_rate.min': 'pulso_min_bpm',
    'com.samsung.health.heart_rate.max': 'pulso_max_bpm',
    'com.samsung.health.heart_rate.time_offset': 'zona_horaria'
})
df_pulso.head()

,tiempo_inicio,tiempo_fin,pulso_bpm,pulso_min_bpm,pulso_max_bpm,zona_horaria
0,2025-05-30 12:50:48.616,2025-05-30 12:50:48.616,85.0,NaN,NaN,UTC-0400
1,2025-05-30 13:00:29.290,2025-05-30 13:00:29.290,82.0,NaN,NaN,UTC-0400
2,2025-05-30 13:11:00.876,2025-05-30 13:11:00.876,98.0,NaN,NaN,UTC-0400
3,2025-05-30 13:30:29.280,2025-05-30 13:30:29.280,82.0,NaN,NaN,UTC-0400
4,2025-05-30 13:40:29.267,2025-05-30 13:40:29.267,78.0,NaN,NaN,UTC-0400


In [21]:
df_pulso.tail(10)

,tiempo_inicio,tiempo_fin,pulso_bpm,pulso_min_bpm,pulso_max_bpm,zona_horaria
1123,2025-06-30 03:40:29.340,2025-06-30 03:40:29.340,74.0,NaN,NaN,UTC-0400
1124,2025-06-30 03:50:29.357,2025-06-30 03:50:29.357,78.0,NaN,NaN,UTC-0400
1125,2025-06-30 04:00:50.941,2025-06-30 04:00:50.941,73.0,NaN,NaN,UTC-0400
1126,2025-06-30 04:10:50.941,2025-06-30 04:10:50.941,95.0,NaN,NaN,UTC-0400
1127,2025-06-30 04:20:29.338,2025-06-30 04:20:29.338,83.0,NaN,NaN,UTC-0400
1128,2025-06-30 04:30:29.338,2025-06-30 04:30:29.338,82.0,NaN,NaN,UTC-0400
1129,2025-06-30 04:40:29.309,2025-06-30 04:40:29.309,85.0,NaN,NaN,UTC-0400
1130,2025-06-30 04:50:29.309,2025-06-30 04:50:29.309,73.0,NaN,NaN,UTC-0400
1131,2025-06-30 19:20:29.368,2025-06-30 19:20:29.368,82.0,NaN,NaN,UTC-0400
1132,2025-06-30 19:30:31.326,2025-06-30 19:30:31.326,86.0,NaN,NaN,UTC-0400


In [1]:
import uuid
from datetime import datetime

def analyze_uuid(uuid_string):
    """Analiza un UUID y extrae información disponible"""
    
    uuid_obj = uuid.UUID(uuid_string)
    
    print(f"🔍 Análisis del UUID: {uuid_string}")
    print(f"📋 Versión: {uuid_obj.version}")
    print(f"🔧 Variante: {uuid_obj.variant}")
    
    if uuid_obj.version == 1:
        # UUID v1 contiene timestamp
        timestamp = uuid_obj.time
        print(f"⏰ Timestamp: {datetime.fromtimestamp(timestamp/1e7 - 12219292800)}")
        print(f"🖥️ Nodo: {hex(uuid_obj.node)}")
    elif uuid_obj.version == 4:
        print("🎲 UUID v4: Generado aleatoriamente")
    
    return uuid_obj

# Analizar tu UUID
your_uuid = "2f52f76d-ec1b-483e-8e72-c34a9212cac1"
analyze_uuid(your_uuid)

🔍 Análisis del UUID: 2f52f76d-ec1b-483e-8e72-c34a9212cac1
📋 Versión: 4
🔧 Variante: specified in RFC 4122
🎲 UUID v4: Generado aleatoriamente


UUID('2f52f76d-ec1b-483e-8e72-c34a9212cac1')